In [ ]:
import hashlib
import numpy as np
import pybryt

# Annotations

To understand how to build a reference implementation, we must first take a look at the building blocks of reference implementations: annotations. **Annotations** are Python objects that an instructor creates to assert conditions on each submission; these can be things like the presence of a value, the temporal relationship between values, or even the time complexity of a block of code. Annotations are created by instantiating classes provided by the `pybryt` package.

There are three main types of annotations:

* _value annotations_, which assert the presence of some value in the student's memory footprint
* _relational annotations_, which assert a relationship between values in the student's memory footprint, founded in boolean logic or temporality
* _complexity annotations_, which assert a time complexity on a block of code

In this module, we will discuss the first.

## Value Annotations

[**Value annotations**](https://microsoft.github.io/pybryt/html/annotations/value_annotations.html) assert the presence of a value in the student's memory footprint. They are created by instantiating the class [`pybryt.Value`](https://microsoft.github.io/pybryt/html/api_reference.html#pybryt.annotations.value.Value), which takes the value you want to look for as its only positional argument.

Consider, for example, that you wanted to check that a student correctly initialized an array. In a reference implementation, you would initialize the array, and then create an instance of the class:

In [ ]:
np.random.seed(42)
arr = np.random.normal(size=(100,100))
pybryt.Value(arr)

All annotations that are created are internally tracked by PyBryt, so there's no need to assign them to a variable unless you want to use them to create more complex annotations (more on that below).

Value annotations, when being used to check numerical values (including arrays, iterables of numbers, and dataframes), also support absolute and relative tolerance using the `atol` and `rtol` arguments:

In [ ]:
value_annotation = pybryt.Value(arr, atol=1e-3)
value_annotation.check_against(arr.round(3)), value_annotation.check_against(arr.round(2))

The method `pybryt.Value.check_against` returns a boolean value indicating whether the object passed to it satisfies the value annotation. As you can see above, by allowing an absolute tolerance of $10^{-3}$, the value was satisfied when the array values were rounded to thre places, but failed when rounded to two places.

### Equivalence Fuctions

While `Value` objects define an algorithm to determine whether two objects are equal, they also allow users to specify a custom equivalence function to be used for the comparison. For example, let's say you wanted to look for a string in the student's code, but were unconcerned about the capitalization of that string. You could use a custom equivalence function that compares the lowercased representation of two strings:

In [ ]:
def str_equal_lower(s1, s2):
    return s1.lower() == s2.lower()

Then, we could use this function to create a `Value` annotation to check hexadecimal strings, for example:

In [ ]:
message = "hash me"
sha1_hash = hashlib.sha1(message.encode()).hexdigest()

sha1_annotation = pybryt.Value(sha1_hash, equivalence_fn=str_equal_lower)
sha1_annotation.check_against(sha1_hash.upper())

## Annotation Options

All annotations support some options that allow you to tailor the feedback your students receive as a result of the passing or failing of those annotations. The two primary methods of providing this feedback are using the `success_message` and `failure_message` arguments in the constructor, available to all annotations:

In [ ]:
v = pybryt.Value(1, success_message="Found 1!", failure_message="Didn't find 1 :(")

If the value is found in the student's memory footprint, the `success_message` will be included in the report that is generated by PyBryt's student implementation checker (more on that later); if it is not found, the `failure_message` will be included. If these are not provided, no message is shown.

These can also be set by updating the correspondingly-named fields in the annotation object:

In [ ]:
v.success_message = "Congrats!"
v.failure_message = "Try again"

## Collection Annotations

The most basic type of relational annotation is the [collection annotation](https://microsoft.github.io/pybryt/html/annotations/collections.html), which simply collects a group of annotations together so they can be operated on as a unit. It is possible to enforce the order of the annotations in the collection (based on insertion order), but this is optional. Like all other annotations, feedback can be provided based on whether the collection is satisfied using the `success_message` and `failure_message` arguments.

To create a collection, instantiate a `pybryt.Collection`. The constructor takes any number of positional arguments, which correspond to the initial set of annotations in the collection. To initialize an empty collection, provide no positional arguments.

In [ ]:
col = pybryt.Collection(pybryt.Value(1))

To enforce the order of annotations in the collection, pass `enforce_order=True`:

In [ ]:
col = pybryt.Collection(pybryt.Value(1), enforce_order=True)

Annotations can be added to the collection using `pybryt.Collection.add`:

In [ ]:
col.add(pybryt.Value(2))

An annotation collection is satisfied when all of its children are satisfied and, if `enforce_order` is true, if the satisfying timestamps of its children occur in non-decreasing order. For example, let's check `col` against two memory footprints: one in which 1 occurs before 2, and the other in which 2 occurs before 1.

In [ ]:
col.check([(1, 1), (2, 2)]), col.check([(2, 1), (1, 2)])